# Welcome to Jina - Let's Deploy an AI service!

## Install requirements

We'll use `%%capture` to capture the output (so we don't fill up the whole notebook with `pip`'s log messages).

In [1]:
%%capture
!pip install jina transformers
# !pip install -U docarray  # Use latest version of DocArray v2

## Write Executor

We'll use Transformer's pipeline to work with StableLM

In [3]:
from jina import Executor, requests
from docarray import DocumentArray

from transformers import pipeline


class StableLM(Executor):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.generator = pipeline('text-generation', model='stablelm-3b')

    @requests
    def generate(self, docs: DocumentArray, **kwargs):
        generated_text = self.generator(docs.texts)
        docs.texts = [gen[0]['generated_text'] for gen in generated_text]

## Create input Document

Everything that goes in and out of Jina is a [Document](), provided by the [DocArray package](). In our Document we'll add a prompt that will be sent to the LLM as input.

In [9]:
input_doc = Document(tags={'prompt':'suggest an interesting image generation prompt for a mona lisa variant'})

## Deploy the AI service to create an image generation prompt

We'll deploy our service and send a prompt. The Deployed Executor will process this in the LLM and return a prompt for image generation.

In [ ]:
from jina import Deployment
from docarray import Document

dep = Deployment(uses=StableLM, timeout_ready=-1, port=12345)

with dep:
    docs = dep.post(on='/', inputs=[input_doc])

Note: Typically you would use something like:

```python
with dep:
    dep.block()
```

To block the process and keep a gateway open. However, since this is a Jupyter notebook we could not then send an input Document. For this reason we use `dep.post()` in this notebook instead.